# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Julia Notebook_

## Kernel creation instructions
1. 

TODO: create python kernel and rearrange some of these cells

In [1]:
using Pkg
Pkg.status()

Status `/projects/a9009/tdm5510/workshops/julia_workshops/devel_2025/environment_toml_test/my_cool_project/Project.toml`
  [8f4d0f93] Conda v1.10.3
  [7073ff75] IJulia v1.32.1
  [91a5bcdd] Plots v1.41.1


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.11.4
Commit 8561cc3d68d (2025-03-10 11:36 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 64 × Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, icelake-server)
Threads: 1 default, 0 interactive, 1 GC (on 64 virtual cores)
Environment:
  LD_LIBRARY_PATH = /.singularity.d/libs


# Runtime Differences Between Python and Julia

In [3]:
# Julia
@time for i in 1:100
    a = rand(1000, 1000)
    b = rand(1000)
    x = a \ b
end

  3.377238 seconds (1.50 k allocations: 1.492 GiB, 31.78% gc time)


In [ ]:
# Python with NumPy
import numpy as np
from time import time



start = time()
for i in range(100):
    a = np.random.rand(1000, 1000)
    b = np.random.rand(1000)
    x = np.linalg.solve(a, b)
end = time()

print(end - start)

In [ ]:
# Python with Numba
from numba import jit
import numpy as np
from time import time

@jit(nopython=True)
def solve_equation(a, b):
    n = len(b)
    x = np.empty(n)

    for i in range(n):
        x[i] = b[i] / a[i, i]
        for j in range(i + 1, n):
            b[j] -= a[j, i] * x[i]

    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            x[i] -= a[i, j] * x[j]
        x[i] /= a[i, i]

    return x

@jit(nopython=True)
def run_code(x):
    for i in range(x):
        a = np.random.rand(1000, 1000)
        b = np.random.rand(1000)
        c = solve_equation(a, b)

start = time()
run_code(100)
end = time()
print((end - start), "seconds")

# Explanation of `methods` in Julia

---



The concept of multiple dispatch is something we use frequently but never really stop to contemplate.  Julia surfaces multiple dispatch in a concrete and easy fashion compared to operator overloading or templating.

In [ ]:
f(a::Int64, b::Int64) = a + b

f(a::Float64, b::Float64) = a * b

f(a::Number, b::Number) = 2 * (a + b)

println(f(2 , 3))
println(f(2.0, 3.0))
println(f(2, 3.0))

methods(f)

In [ ]:
methods(+)

# Linear Regression - Julia


In [ ]:
import Pkg
Pkg.add("Plots")
Pkg.add("GLM")
Pkg.add("DataFrames")
using DataFrames
using BenchmarkTools
using GLM
using Plots

# perform the ordinary least squares fits
data1 = DataFrame(X1=collect(0:100), Y1=rand(0:100,101))
data2 = DataFrame(X2=collect(0:1000), Y2=rand(0:100,1001))
data3 = DataFrame(X3=collect(0:10000), Y3=rand(0:100,10001))
data4 = DataFrame(X4=collect(0:100000), Y4=rand(0:100,100001))

ols1 = @btime lm(@formula(Y1 ~ X1), data1)
ols2 = @btime lm(@formula(Y2 ~ X2), data2)
ols3 = @btime lm(@formula(Y3 ~ X3), data3)
ols4 = @btime lm(@formula(Y4 ~ X4), data4)

# plot the points
plot(data1.X1, data1.Y1, linewidth=2, title="X vs Y", label=["Y"], xlabel="X", ylabel="Y")

# display the coefficients
println(coef(ols1))

Yp = predict(ols1); #load predictions into Yp
Yn = Base.hcat(data1.Y1,Yp) #concatenating Y and Yp arrays into a new one called 'Yn' so that we can plot both, below
plot(data1.X1, Yn, linewidth=2, title="X vs Y", label=["Y", "Yp"], xlabel="X", ylabel="Y")

# Linear Regression - Python

In [ ]:
import statsmodels.api as sm
import numpy as np
import time

# defining the variables
x1 = list(range(101))
x2 = list(range(1001))
x3 = list(range(10001))
x4 = list(range(100001))
y1 = np.random.uniform(low=0, high=100, size=(101,))
y2 = np.random.uniform(low=0, high=100, size=(1001,))
y3 = np.random.uniform(low=0, high=100, size=(10001,))
y4 = np.random.uniform(low=0, high=100, size=(100001,))

# performing the regression
# and fitting the model
starttime = time.perf_counter()
result = sm.OLS(y1, x1).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y2, x2).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y3, x3).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y4, x4).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

# printing the summary table
print(result.summary())

# Introduction to DataFrames in Julia

##### Version 0.1

***

By Scott Coughlin (Northwestern IT Research Computing and Data Services)  
30 November 2023

### Installing and Loading Packages for Julia DataFrames

*First*, we need to install the DataFrames package from Julia.

In [ ]:
using Pkg
Pkg.add(["DataFrames","CSV","Query"])

Now that we have installed the DataFrames package, we need load it.


In [ ]:
# import pandas
using DataFrames, CSV, Query

## Creating a DataFrame

As with Pandas, there are many ways to construct a DataFrame in Julia. Below, we will go through some examples and comparisons.







### Standard Construction of a DataFrame

As with Pandas, there are many ways to construct a DataFrame in Julia. Below, we will go through some examples and comparisons.


In [ ]:
# df = pandas.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
#                    columns=['a', 'b', 'c'])

# Pass column names as strings
df = DataFrame([1 2 3; 4 5 6; 7 8 9], ["a", "b", "c"])

# Pass column names as "Symbols"
df2 = DataFrame([1 2 3; 4 5 6; 7 8 9], [:a, :b, :c])
print(df)
print(df2)

One very important thing to note in this above is the syntax is the definition of the column names. In Python, if you are going to define a string, you can use either double or single quotes. However, Julia will not like column names in single quotes. The cell below will fail.

In [ ]:
df = DataFrame([1 2 3; 4 5 6; 7 8 9], ['a', 'b', 'c'])

I think this is probably the best explanation: https://stackoverflow.com/questions/23480722/what-is-a-symbol-in-julia


### From a Dictionary

In [ ]:
# df = pandas.DataFrame({"customer_age" : [15, 20, 25], "first_name" : ["Scotty", "Matthew", "Sophie"]})

dict1 = Dict("customer_age" => [15, 20, 25],
                   "first_name" => ["Scotty", "Matthew", "Sophie"])

dict2 = Dict(:customer_age => [15, 20, 25],
                   :first_name => ["Scotty", "Matthew", "Sophie"])

df1 = DataFrame(dict1)
df2 = DataFrame(dict2)
print(df1)
print(df2)

### From a CSV File

First, we need to download a CSV file locally. We will actually be using the CSV file below later on in our exercises section, so let us download it now to get a head start!

In [ ]:
# Make a call out to the "shell" to use `wget` to download the CSV file
run(pipeline(`wget -nv https://raw.githubusercontent.com/nuitrcs/Julia_workshop/master/IMDB-directors.csv -O /tmp/IMDB-directors.csv`))

In [ ]:
# pandas.read_csv("/tmp/IMDB-directors.csv")
input_file = joinpath("/tmp", "IMDB-directors.csv");
# One way
df1 = DataFrame(CSV.File(input_file));
# Another way
df2 = CSV.read(input_file, DataFrame);

## Joining Data Frames

In [ ]:
dict1 = Dict("customer_id" => [1, 2, 3],
             "first_name" => ["Scotty", "Matthew", "Sophie"])
dict2 = Dict("customer_id" => [2, 3, 4],
             "last_name" => ["Gorby", "van Genderen", "Kinaci"])
dict3 = Dict("customer_id" => [1, 2, 3],
             "home" => ["Minnesota", "Oregon", "Netherlands"])


df1 = DataFrame(dict1);
df2 = DataFrame(dict2);
df3 = DataFrame(dict3);

#### `innerjoin`

The resulting DataFrame will contain rows from each DataFrame where the value of the "on" column is found in both DataFrames



In [ ]:
print(innerjoin(df1, df3, on = :customer_id))
print(innerjoin(df1, df2, on = :customer_id))

#### `outerjoin`
The resulting DataFrame will contain all rows from both DataFrames. A value of "missing" will be added for certain row, column values for column and row combinations not found in one or the other DataFrame.



In [ ]:
print(outerjoin(df1, df2, on = :customer_id))

#### `rightjoin`
The joining and resulting rows will be based on the rows from the second DataFrame passed to the function

In [ ]:
print(rightjoin(df1, df2, on = :customer_id))

#### `leftjoin`
The joining and resulting rows will be based on the rows from the first DataFrame passed to the function

In [ ]:
print(leftjoin(df1, df2, on = :customer_id))

#### `semijoin`
The returned DataFrame will be only the rows of the first passed DataFrame which overlap with the "on" column of the second passed DataFrame


In [ ]:
print(semijoin(df1, df2, on = :customer_id))

#### `antijoin`
The returned DataFrame will be only the rows of the first passed DataFrame which *do not* overlap with the "on" column of the second passed DataFrame


In [ ]:
print(antijoin(df1, df2, on = :customer_id))

## Performing Selections and Queries on DataFrames

In [ ]:
# pandas.read_csv("/tmp/IMDB-directors.csv")
input_file = joinpath("/tmp", "IMDB-directors.csv");
df = CSV.read(input_file, DataFrame);

In [ ]:
# Select rows 1, 6, 10
df[[1, 6, 10], :]

In [ ]:
# Select all rows but only column `first_name`
df[:, [:first_name]]

In [ ]:
# Select all rows but only column `first_name` but return an Vector
df[:, :first_name]

In [ ]:
# Flip the DataFrame and return only columns director_id and first_name
@view df[end:-1:1, [:director_id, :first_name]]

In [ ]:
# Add a column called "Movie Good", and make all the values be the string "Awesome"
df[!, :movie_good] .= "Awesome";
df

In [ ]:
# Select all rows where first_name is Todd
df[(df[!,:first_name].=="Todd"),:]

In [ ]:
# Select all rows where first_name is Todd using the Query package
q1 = @from i in df begin
            @where i.first_name == "Todd"
            @select {i.first_name, i.director_id, i.last_name}
            @collect DataFrame
       end

In [ ]:
# Select all rows where first_name is Todd and last_name is Barry
df[.&(df[!,:first_name].=="Todd", df[!,:last_name].=="Barry"),:]

In [ ]:
# Select all rows where first_name is Todd and last_name is Barry using the Query package
q1 = @from i in df begin
            @where i.first_name == "Todd" && i.last_name == "Barry"
            @select {i.first_name, i.director_id, i.last_name}
            @collect DataFrame
       end

## Exercise: IMDb Data
Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding databases.

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb (there are no movies after 2004).

First, we need to download all of the CSV's that we plan on using.

In [ ]:
run(pipeline(`wget -nv https://raw.githubusercontent.com/nuitrcs/Julia_workshop/master/IMDB-movies.csv -O /tmp/IMDB-movies.csv`))
run(pipeline(`wget -nv https://raw.githubusercontent.com/nuitrcs/Julia_workshop/master/IMDB-movies_directors.csv -O /tmp/IMDB-movies_directors.csv`))
run(pipeline(`wget -nv https://raw.githubusercontent.com/nuitrcs/Julia_workshop/master/IMDB-movies_genres.csv -O /tmp/IMDB-movies_genres.csv`))

Below we load in the necessary data from CSV files and create 4 DataFrames where we print the first 5 rows of each.

In [ ]:
imdb_movies = CSV.read(joinpath("/tmp", "IMDB-movies.csv"), DataFrame);
imdb_directors = CSV.read(joinpath("/tmp", "IMDB-directors.csv"), DataFrame);
imdb_movies_directors = CSV.read(joinpath("/tmp", "IMDB-movies_directors.csv"), DataFrame);
imdb_movies_genres = CSV.read(joinpath("/tmp", "IMDB-movies_genres.csv"), DataFrame);
print(first(imdb_movies, 5))
print(first(imdb_directors, 5))
print(first(imdb_movies_directors, 5))
print(first(imdb_movies_genres, 5))

Let's look at what one would do to combine the information about Movie Titles and Years with information about Movie Genres

In [ ]:
innerjoin(imdb_movies_genres, imdb_movies, on=:movie_id)

To make things simple for the rest of the notebook, let's *join* some of these dataframes together in order to have one DataFrame with information from both `imdb_movies` and `imdb_directories` together in a single DataFrame called `imdb_movies_directors` and extend that logic to making `imdb_movies_directors_genres` and `imdb_movies_genres`.

In [ ]:
## This is what it would look like to do this with Pandas.
#imdb_movies_directors_genres = imdb_movies_genres.merge(imdb_movies).merge(imdb_movies_directors).merge(imdb_directors)
#imdb_movies_genres = imdb_movies_genres.merge(imdb_movies)
#imdb_movies_directors = imdb_movies_directors.merge(imdb_movies).merge(imdb_directors)

## How would you do it in Julia??
imdb_movies_directors_genres =
imdb_movies_genres =
imdb_movies_directors =

For our exercises, there are now 5 Julia DataFrames,
```
imdb_movies
imdb_directors
imdb_movies_directors
imdb_movies_genres
imdb_movies_directors_genres
```

## Problem 1) Simple Queries

**Problem 1a**

* SELECT 10 movies from the imbd_movies table
* Select 10 directors from imbd_directors and order by `first_name`.

*write your answer here*

**Problem 1b**

* How many movies are there?
* How many directors are there?

*write your answer here*

**Problem 1c**

Determine how many movies are there after the year 2000?

*write your answer here*

**Problem 1d**

How many different movie genres are there?

*write your answer here*

## Problem 2) Groups and Aggregates

**Problem 2a**

In which year were the most movies made according to IMDb?

*write your answer here*

**Problem 2b**

How many "Action" movies where made after the year 1980? Before the year 1980?

*write your answer here*

**Problem 2c**

Select all films made by `Scorsese`. How many are there?

*write your answer here*

**Problem 2d**

According the the IMDb data, which director has directed the most movies?

*write your answer here*

**Problem 2e**

According the the IMDb data, which director has directed the most movies in each genre?

*write your answer here*